In [12]:
import numpy as np
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification
)
from datasets import load_from_disk
import evaluate
import torch

In [4]:
train_dataset = load_from_disk("data/preprocessed/train_processed")
test_dataset = load_from_disk("data/preprocessed/test_processed")

In [10]:
# tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
# model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

model.save_pretrained("models/distilbert-base-uncased")

In [11]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained('models/distilbert-base-uncased', num_labels=2)

In [13]:
def predict(input_data, model, tokenizer):    
    inputs = tokenizer(
        input_data, 
        return_tensors="pt", 
        padding=True, 
        truncation=True, 
        max_length=256
    )
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Return probabilities and predicted class
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    print(f'probs: {probs}')
    pred_class = torch.argmax(probs).item()
    print(f'pred_class: {pred_class}')
    return {"prediction": pred_class, "probabilities": probs.tolist()[0]}

print(predict("Hello, this is a test message.", model, tokenizer))

probs: tensor([[0.5162, 0.4838]])
pred_class: 0
{'prediction': 0, 'probabilities': [0.5161833167076111, 0.48381659388542175]}


In [ ]:
# Metrics
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)